# calculate rotation and transposition matrix

In [1]:
import os

data_folder = r'D:\Pu\20211206-P_brain_CTP12-RNA'
#before_position_file = os.path.join(data_folder, 'positions_before_align.txt')
before_position_file = os.path.join(data_folder, '10x_position_before.txt')
#after_position_file = os.path.join(data_folder, 'positions_after_align.txt')
after_position_file = os.path.join(data_folder, '10x_position_after.txt')

In [2]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [3]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [4]:
R, T

(array([[ 0.9999574 , -0.00922978],
        [ 0.00922978,  0.9999574 ]]),
 array([-1511.41002712,   860.9771437 ]))

# transpose 60x positions

In [5]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [6]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-7183.53133535 -3688.39419943]
 [-7181.68537911 -3488.4027185 ]
 [-7179.83942286 -3288.41123757]
 [-7177.99346661 -3088.41975663]
 [-7176.14751036 -2888.4282757 ]
 [-7376.13899129 -2886.58231945]
 [-7377.98494754 -3086.57380038]
 [-7379.83090379 -3286.56528132]
 [-7381.67686004 -3486.55676225]
 [-7383.52281629 -3686.54824318]
 [-7385.36877254 -3886.53972411]
 [-7387.21472879 -4086.53120505]
 [-7589.05216597 -4284.67672973]
 [-7587.20620972 -4084.6852488 ]
 [-7585.36025347 -3884.69376786]
 [-7583.51429722 -3684.70228693]
 [-7581.66834097 -3484.710806  ]
 [-7579.82238472 -3284.71932507]
 [-7577.97642847 -3084.72784413]
 [-7576.13047222 -2884.7363632 ]
 [-7776.12195315 -2882.89040695]
 [-7777.9679094  -3082.88188788]
 [-7779.81386565 -3282.87336882]
 [-7781.6598219  -3482.86484975]
 [-7783.50577815 -3682.85633068]
 [-7785.3517344  -3882.84781161]
 [-7787.19769065 -4082.83929255]
 [-7789.0436469  -4282.83077348]
 [-7989.03512783 -4280.98481723]
 [-7987.18917158 -4080.9933363 ]
 [-7985.34

In [7]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20211206-P_brain_CTP12-RNA\translated_positions_all.txt


# further adjust manually

In [40]:
before_adjust = np.array([[-7177.99,-3088.42],
                         [-7735.65,777.94],
                         [-886.44,-2109.92],
                         [-3666.17,1486.88]])


after_adjust = np.array([[-7320.0,-3137.5],
                        [-7882.0,728.0],
                        [-1027.0,-2147.5],
                        [-3813.7,1443.0]])


after_adjust -before_adjust

array([[-142.01,  -49.08],
       [-146.35,  -49.94],
       [-140.56,  -37.58],
       [-147.53,  -43.88]])

In [43]:
manual_shift = np.mean(after_adjust -before_adjust, axis=0)
manual_shift

array([-144.1125,  -45.12  ])

In [44]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [45]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Pu\20211206-P_brain_CTP12-RNA\adjusted_translated_positions_all.txt
